In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

#preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import mode


# pipelines
from sklearn.pipeline import Pipeline



# Customer Data

In this assignment we are going to examine some customer data gathering from my very famous internet company "the best one ever".  "The best one ever" is the best company ever that sells important things online.  In this investigation we want to find if there are any natural groups of customers in my dataset.  The first step is to just get the data in a format we can feed to our machine learning models. Once we do that, then our boss (some dude named Gilad), said he will teach us how to cluster the customers! But it turns out you need to have the _data_ formatted in some special way...? Maybe you can tell me about that!

In [ ]:
# Read in the data found in "best_one_ever_database.csv" and take a look at the head and info.

In [5]:
data = pd.read_csv('best_one_ever_database.csv', index_col='id')

In [6]:
data.head()

,first_name,last_name,email,gender,ip_address,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent
id,,,,,,,,,,
1,Jarred,Hallsworth,jhallsworth0@biblegateway.com,Non-Binary,223.109.240.148,$606.16,85349,7,NaN,3310.505486
2,Wayne,Robez,wrobez1@t.co,Male,73.174.113.87,$147.22,85336,9,NaN,2188.852818
3,Casper,Hayles,chayles2@cdc.gov,Male,223.249.117.173,$801.97,85350,4,0.597445,2640.499644
4,Mandie,Szepe,mszepe3@lulu.com,Female,31.143.95.228,$336.77,85364,4,NaN,2872.580992
5,Noam,Cecely,ncecely4@elegantthemes.com,Male,63.71.174.230,$757.45,85369,8,0.127835,2728.008966


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1 to 2000
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   first_name                 2000 non-null   object 
 1   last_name                  2000 non-null   object 
 2   email                      2000 non-null   object 
 3   gender                     2000 non-null   object 
 4   ip_address                 2000 non-null   object 
 5   sales                      2000 non-null   object 
 6   zip_code                   2000 non-null   int64  
 7   total_num_of_transactions  2000 non-null   int64  
 8   prob_of_rebuy              1538 non-null   float64
 9   money_spent                1690 non-null   float64
dtypes: float64(2), int64(2), object(6)
memory usage: 171.9+ KB


# Cleaning The Data
# Cleaning The Data

Taking a look at the data you have to ask yourself the questions

1. 'Which columns are useful for me to keep?'
2. 'Are all the columns usable as features?

Then you may have to do some "work" to get the column to be usable. Let's look at one column together. The first column is titled "first_name" and it seems to be the first name of each customer. Is this a usable feature? Well... not exactly in string format. So I guess I could one hot encode them into binary vectors, but even then... do I want to cluster the customers based on their first name? You can imagine some situation where clustering by name might be relevant (for example trying to guess what generation someone belonged to?) but in this case it seems like it's more of a unique identifier so it may be best to simply remove it. If every value in a column is unique (there are no duplications of the value) then we shouldn't use it as a feature because it will have a 1-1 mapping with the target variable which is not something we ever want. We want our model to learn and generalize from the features, not memorize that the name "jane" bought 5 cans of soda.

Ok, that's the first column, we vote drop! Now you have to go through each and every column and ask yourself "do I keep it? if yes, what extra work might I have to do?" 

Let's walk through it
  
  1. first_name:  this is a unique identifier so we should remove it.
  2. last_name: see above
  3. email: this is unique to an extent.  BUT if we strip the name@ portion of the email and simply keep the domain name, it could possibly aid us. Perhaps certain kinds of customers use certain email services! Worth looking into
  4. Gender: this is certainly relevant, but it's categorical data. We will need to one-hot-encode it.
  5. ip_address: We can perhaps segment the ip's into fields and use them, there maybe overlaps or correlations among different fields. Or maybe you know more about IP addresses than I do and this is totally useless
  6. sales: we certainly need this column, but we need to convert it a floating point type: remove the '$' and convert the dtype of the column
  7. zip_code: I think we can just leave this as is.
  8. prob-of_rebuy: I think we can just leave this as is.
  9. Money_spent: seems fine to me!
  

In [42]:
# drop the name columns using pandas.drop()
X = data.drop(columns=['first_name','last_name']) # fill in the parameters

In [43]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1 to 2000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   email                      2000 non-null   object 
 1   gender                     2000 non-null   object 
 2   ip_address                 2000 non-null   object 
 3   sales                      2000 non-null   object 
 4   zip_code                   2000 non-null   int64  
 5   total_num_of_transactions  2000 non-null   int64  
 6   prob_of_rebuy              1538 non-null   float64
 7   money_spent                1690 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 205.2+ KB


In [41]:
# always reasonable to check `.info()` or .head() after an operation to make sure it worked
# how you thought, at least until you are comfortable with the methods / functions
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1 to 2000
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   email                      2000 non-null   object 
 1   gender                     2000 non-null   object 
 2   ip_address                 2000 non-null   object 
 3   sales                      2000 non-null   object 
 4   zip_code                   2000 non-null   int64  
 5   total_num_of_transactions  2000 non-null   int64  
 6   prob_of_rebuy              1538 non-null   float64
 7   money_spent                1690 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 205.2+ KB


### Transform string columns into useful features

1. email column
2. sales column (remove $ sign)

We will use the pandas `apply` function take a function that operates on a string and apply it to the entire column. I will do the first one, and you will do the next one.

In [44]:
def strip_dollar(x):
    return x.strip('$')

In [53]:
X.sales = X.sales.apply(strip_dollar).astype('Float32')

SyntaxError: invalid decimal literal (2454387294.py, line 1)

In [50]:
X.sales

id
1       606.159973
2       147.220001
3       801.969971
4       336.769989
5       757.450012
           ...    
1996    452.619995
1997    485.459991
1998    658.119995
1999    716.780029
2000    361.769989
Name: sales, Length: 2000, dtype: Float32

In [39]:
# for i in range(1,len(X.sales)+1):
#     X.sales[i] = (X.sales[i]).replace("$","")


C:\Users\13392\AppData\Local\Temp\ipykernel_11400\1030197387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sales[i] = (X.sales[i]).replace("$","")


In [52]:
X.head()

,email,gender,ip_address,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent
id,,,,,,,,
1,jhallsworth0@biblegateway.com,Non-Binary,223.109.240.148,606.159973,85349,7,NaN,3310.505486
2,wrobez1@t.co,Male,73.174.113.87,147.220001,85336,9,NaN,2188.852818
3,chayles2@cdc.gov,Male,223.249.117.173,801.969971,85350,4,0.597445,2640.499644
4,mszepe3@lulu.com,Female,31.143.95.228,336.769989,85364,4,NaN,2872.580992
5,ncecely4@elegantthemes.com,Male,63.71.174.230,757.450012,85369,8,0.127835,2728.008966


### Your Turn

Now you need to
1. write a function to strip the name portion of the email
2. Apply it to the column


In [56]:
# define functions to apply to the dataframe
def strip_emails(x):
    # your code here
    loc = x.find('@')
    return x[loc+1:]

test_email = 'thisismymail@gmail.com'
print(strip_emails(test_email))

gmail.com


In [57]:
# apply the function to the column and assign it back to the column (it does not work inplace)
X.email = X.email.apply(strip_emails)
X.email

id
1        biblegateway.com
2                    t.co
3                 cdc.gov
4                lulu.com
5       elegantthemes.com
              ...        
1996       opensource.org
1997            yahoo.com
1998           usnews.com
1999           toplist.cz
2000        woothemes.com
Name: email, Length: 2000, dtype: object

# Is the email column going to be worth it?
Let's take a look at this email column and decide if it could help us or not.


In [59]:
# how many unique domains are there?
counts = X.email.value_counts()
counts

tiny.cc                10
xrea.com               10
furl.net               10
qq.com                 10
cbsnews.com            10
                       ..
fda.gov                 1
ed.gov                  1
devhub.com              1
creativecommons.org     1
ox.ac.uk                1
Name: email, Length: 490, dtype: int64

Based on what you think, you can either keep the column or not. It never hurts to try using it and then drop it later, it's also not a big deal if you are sure it's going to be useless. You will learn to have intuition with these things over time.

In [61]:
X.drop(columns='email', inplace=True)
X

,gender,ip_address,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent
id,,,,,,,
1,Non-Binary,223.109.240.148,606.159973,85349,7,NaN,3310.505486
2,Male,73.174.113.87,147.220001,85336,9,NaN,2188.852818
3,Male,223.249.117.173,801.969971,85350,4,0.597445,2640.499644
4,Female,31.143.95.228,336.769989,85364,4,NaN,2872.580992
5,Male,63.71.174.230,757.450012,85369,8,0.127835,2728.008966
...,...,...,...,...,...,...,...
1996,Female,99.212.77.78,452.619995,84720,9,NaN,2864.652072
1997,Female,43.253.114.105,485.459991,93424,2,0.125885,3432.813049
1998,Male,70.249.185.148,658.119995,93424,10,0.249830,2406.382405


## Splitting the IP Address
We now need to split up the IP address, we will use Pandas's built in str method for this.
Again, I'm not even sure this is a good idea, but it's certainly fine practice to learn how to spit up strings in pandas!

In [72]:
X['ip_address'].str[:]

id
1       223.109.240.148
2         73.174.113.87
3       223.249.117.173
4         31.143.95.228
5         63.71.174.230
             ...       
1996       99.212.77.78
1997     43.253.114.105
1998     70.249.185.148
1999    202.103.209.127
2000     241.21.156.112
Name: ip_address, Length: 2000, dtype: object

In [73]:
# We are going to use pandas str method here, pay close attention!

X[['first_ip','second_ip','third_ip','fourth_ip']] = X.ip_address.str.split(pat=".", expand=True)

In [ ]:
# now we cast the columns as floats, because we always need numbers for our models!
X[['first_ip','second_ip','third_ip','fourth_ip']] = X[['first_ip','second_ip','third_ip','fourth_ip']].astype('int')
# we also drop the original column
X.drop(columns='ip_address', inplace=True)

In [89]:
X.head()

,gender,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent,first_ip,second_ip,third_ip,fourth_ip,Female,Male,Non-Binary
id,,,,,,,,,,,,,
1,Non-Binary,606.159973,85349,7,NaN,3310.505486,223,109,240,148,0,0,1
2,Male,147.220001,85336,9,NaN,2188.852818,73,174,113,87,0,1,0
3,Male,801.969971,85350,4,0.597445,2640.499644,223,249,117,173,0,1,0
4,Female,336.769989,85364,4,NaN,2872.580992,31,143,95,228,1,0,0
5,Male,757.450012,85369,8,0.127835,2728.008966,63,71,174,230,0,1,0


# One Hot Encoding

Ok we are almost done, we just have to convert the gender column into something integer that we can use. We will use one-hot-encoding since gender is a categorical variable.

Pandas has a `get_dummies()` function that will be very useful.
I'm going to let you look it up and learn how to use it.


In [79]:
# check the gender distribution, just because
X.gender.value_counts()

Male          775
Female        725
Non-Binary    500
Name: gender, dtype: int64

In [90]:
gender_df = pd.get_dummies(X.gender)
gender_df

,Female,Male,Non-Binary
id,,,
1,0,0,1
2,0,1,0
3,0,1,0
4,1,0,0
5,0,1,0
...,...,...,...
1996,1,0,0
1997,1,0,0
1998,0,1,0


In [92]:
X=X.join(gender_df)
X.drop(columns='gender', inplace=True)

In [98]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1 to 2000
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sales                      2000 non-null   Float32
 1   zip_code                   2000 non-null   int64  
 2   total_num_of_transactions  2000 non-null   int64  
 3   prob_of_rebuy              1538 non-null   float64
 4   money_spent                1690 non-null   float64
 5   first_ip                   2000 non-null   int32  
 6   second_ip                  2000 non-null   int32  
 7   third_ip                   2000 non-null   int32  
 8   fourth_ip                  2000 non-null   int32  
 9   Female                     2000 non-null   uint8  
 10  Male                       2000 non-null   uint8  
 11  Non-Binary                 2000 non-null   uint8  
dtypes: Float32(1), float64(2), int32(4), int64(2), uint8(3)
memory usage: 189.5 KB


In [103]:
X.isna().sum()

sales                          0
zip_code                       0
total_num_of_transactions      0
prob_of_rebuy                462
money_spent                  310
first_ip                       0
second_ip                      0
third_ip                       0
fourth_ip                      0
Female                         0
Male                           0
Non-Binary                     0
dtype: int64

# Data Preprocessing Stage 2

Ok, we are done with stage 1 - we have converted everything into numeric features and dropped all the unneccessary things. Please double check that! Make sure all features are numeric (a check with .info() should do the trick).

However we do have missing values. Which two columns have missing values?
How many values are missing?
What should we do about those missing values?

You can either impute (fill in) the missing values, or drop the rows which contain them. The choice is up to you!
Either way, you should practice both methods. This way you can practice coding both solutions.

Note:
The `DataFrame.fillna()` method essentially assumes that you are using timeseries data. We are not, so I wouldn't use this. In order to impute simple values, you can use numpy easily, but... I'm lazy and would probably use the scikit-learn implementation.

https://scikit-learn.org/stable/modules/impute.html

In [106]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan, strategy='mean')

In [115]:
X_ = imputer.fit_transform(X)

In [109]:
X = pd.DataFrame(X_, columns=X.columns)

In [111]:
X.head()

,sales,zip_code,total_num_of_transactions,prob_of_rebuy,money_spent,first_ip,second_ip,third_ip,fourth_ip,Female,Male,Non-Binary
0,606.159973,85349.0,7.0,0.510104,3310.505486,223.0,109.0,240.0,148.0,0.0,0.0,1.0
1,147.220001,85336.0,9.0,0.510104,2188.852818,73.0,174.0,113.0,87.0,0.0,1.0,0.0
2,801.969971,85350.0,4.0,0.597445,2640.499644,223.0,249.0,117.0,173.0,0.0,1.0,0.0
3,336.769989,85364.0,4.0,0.510104,2872.580992,31.0,143.0,95.0,228.0,1.0,0.0,0.0
4,757.450012,85369.0,8.0,0.127835,2728.008966,63.0,71.0,174.0,230.0,0.0,1.0,0.0


In [117]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sales                      2000 non-null   float64
 1   zip_code                   2000 non-null   float64
 2   total_num_of_transactions  2000 non-null   float64
 3   prob_of_rebuy              2000 non-null   float64
 4   money_spent                2000 non-null   float64
 5   first_ip                   2000 non-null   float64
 6   second_ip                  2000 non-null   float64
 7   third_ip                   2000 non-null   float64
 8   fourth_ip                  2000 non-null   float64
 9   Female                     2000 non-null   float64
 10  Male                       2000 non-null   float64
 11  Non-Binary                 2000 non-null   float64
dtypes: float64(12)
memory usage: 187.6 KB
